# Starbucks Capstone Challenge

In [18]:
import pandas as pd
import numpy as np
import math
import json
import sqlite3
%matplotlib inline

# Machine Learning & Preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

import pickle

# Metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report
)

# SMOTE for imbalance
from imblearn.over_sampling import SMOTE

# Statsmodels for VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [19]:
# Connect to SQLite database file
conn = sqlite3.connect("df_encoded.db")

# Load the table into a DataFrame
df = pd.read_sql("SELECT * FROM df_encoded", conn) 

# Close the connection
conn.close()

# Check
df.head()

,offer_id,customer_id,completed_flag_final,age,became_member_on,income,reward,difficulty,duration,income_missing,...,gender_M,gender_O,gender_Unknown,"channels_email,mobile,social,web","channels_email,mobile,web","channels_email,web",offer_type_discount,offer_type_informational,income_band_Low,income_band_Medium
0,0b1e1539f2cc45b7b9fa7c272da2e1d7,003d66b6608740288d6cc97a6903f4f0,0.0,26,20170621,73000.0,5,20,10,0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
1,0b1e1539f2cc45b7b9fa7c272da2e1d7,00426fe3ffde4c6b9cb9ad6d077a13ea,0.0,19,20160809,65000.0,5,20,10,0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
2,0b1e1539f2cc45b7b9fa7c272da2e1d7,0056df74b63b4298809f0b375a304cf4,0.0,54,20160821,91000.0,5,20,10,0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,0b1e1539f2cc45b7b9fa7c272da2e1d7,0063def0f9c14bc4805322a488839b32,0.0,118,20180615,63000.0,5,20,10,1,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,0069a50874d846438e58acff5e594725,0.0,118,20160503,63000.0,5,20,10,1,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0


In [20]:
portfolio.head()

NameError: name 'portfolio' is not defined


**profile.json**
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* age (int) - age of the customer
* id (str) - customer id
* became_member_on (int) - date when customer created an app account
* income (float) - customer's income

In [ ]:
profile.head()

**transcript.json**
* person (str) - customer id
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* value - (dict of strings) - either an offer id or transaction amount depending on the record
* time (int) - time in hours since start of test. The data begins at time t=0

In [ ]:
transcript.head()

In [ ]:
# We have multiple IDs, im going to rename them so its easier to see which relates to offer and which to customer for future joins.
portfolio.rename(columns={'id': 'offer_id'}, inplace=True)
profile.rename(columns={'id': 'customer_id'}, inplace=True)
transcript.rename(columns={'person': 'customer_id'}, inplace=True)

In [ ]:
# View them again
portfolio.head()

In [ ]:
profile.head()

In [ ]:
transcript.head()

In [ ]:
transcript.tail()

In [ ]:
# Grouping and viewing unique event values
event_counts = transcript['event'].value_counts()
print(event_counts)

In [ ]:
# Cleaning the value column - seperating it so we know the offer id to join if needed
# Extracting offer_id from both keys into a single column
transcript['offer_id'] = transcript['value'].apply(
    lambda x: x.get('offer id') if isinstance(x, dict) and 'offer id' in x else (
        x.get('offer_id') if isinstance(x, dict) and 'offer_id' in x else None
    )
)

# Create column with amount
transcript['amount'] = transcript['value'].apply(lambda x: x.get('amount') if isinstance(x, dict) else 0)

In [ ]:
# Checking for nulls in my new fields, offer_id and amount
transcript.info()

In [ ]:
# Viewing rows where both offer_id and amount are null - should be none
null_rows = transcript[(transcript['offer_id'].isnull()) & (transcript['amount'].isnull())]
print(null_rows)


## Feature Selection
Features I expect to predict well are:
* age 
* income 
* offer type
* membership duration

In [31]:
# checking for Multicollinearity
# Using the Variance Inflation Factor (VIF) to remove redundant features.
feature_columns = df.columns.tolist()
X = df[feature_columns].copy()
X = X.drop(columns=['offer_id', 'customer_id', 'completed_flag_final'])

X = add_constant(X)  # Add constant term

vif_data = pd.DataFrame()

vif_data["feature"] = X.columns

vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data.sort_values(by="VIF", ascending=False))

/opt/anaconda3/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1783: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
/opt/anaconda3/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/opt/anaconda3/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1783: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


                             feature        VIF
7                     income_missing        inf
11                    gender_Unknown        inf
5                         difficulty  36.976767
4                             reward  18.531901
2                   became_member_on  17.606128
8                        member_days  17.603461
15               offer_type_discount  14.785064
14                channels_email,web   8.636630
17                   income_band_Low   8.275170
16          offer_type_informational   8.074664
3                             income   5.843978
6                           duration   5.591956
18                income_band_Medium   4.150879
1                                age   3.079204
12  channels_email,mobile,social,web   2.974526
13         channels_email,mobile,web   2.558128
9                           gender_M   1.256156
0                              const   0.000000
10                          gender_O        NaN


### Feature	VIF	Interpretation
* income_missing	inf	perfect multicollinearity — likely redundant or derived from another column like income.
* gender_Unknown	inf	Same as above — could be collinear with other gender dummies.
* difficulty, reward, became_member_on, member_days, offer_type_discount	> 10	Very high VIFs — high multicollinearity, likely need investigation.
* Several others	5–10	Moderate multicollinearity — may or may not be problematic depending on the context.
* const	0.0	This is model intercept — always included and not a concern.
* gender_O	NaN	Likely has zero variance (i.e., all 0s), so it's not useful in regression.

#### What is this telling me?

* income_missing        inf - Likey derived from income_bands. Remove.
* gender_Unknown        inf - Likely derived from another column. Remove.
* difficulty  36.976767 - Remove
* reward  18.531901 - Remove, could be derived from offer_type_discount.
* became_member_on  17.606128 - Likely derived from member_days columns. Remove this one.
* member_days  17.603461 - Likely derived from became_member_on columns. Keep this one.
* offer_type_discount  14.785064

#### The columns I am going to drop are:
* income_missing
* gender_Unknown
* difficulty
* reward
* became_member_on


In [36]:
# Drop features with infinite or very high VIFs
cols_to_drop = [
    'income_missing',
    'gender_Unknown',
    'difficulty',
    'reward',
    'became_member_on'
]

df.drop(columns=cols_to_drop, inplace=True)

In [37]:
# Run again to check
# checking for Multicollinearity
# Using the Variance Inflation Factor (VIF) to remove redundant features.
feature_columns = df.columns.tolist()
X = df[feature_columns].copy()
X = X.drop(columns=['offer_id', 'customer_id', 'completed_flag_final'])

X = add_constant(X)  # Add constant term

vif_data = pd.DataFrame()

vif_data["feature"] = X.columns

vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data.sort_values(by="VIF", ascending=False))

/opt/anaconda3/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1783: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


                             feature         VIF
0                              const  198.662837
12                   income_band_Low    8.264871
2                             income    5.794866
3                           duration    4.926045
13                income_band_Medium    3.960772
11          offer_type_informational    2.742368
10               offer_type_discount    2.644197
7   channels_email,mobile,social,web    2.635196
9                 channels_email,web    2.163119
8          channels_email,mobile,web    1.912332
1                                age    1.324557
5                           gender_M    1.211023
4                        member_days    1.002708
6                           gender_O         NaN


In [38]:
df.gender_O.value_counts()

gender_O
0.0    61460
Name: count, dtype: int64

#### Im also going to remove:
* income, looks like income and income bands are derived from each other which they are.
* Remove gender_O

In [40]:
# Drop features with infinite or very high VIFs
cols_to_drop = [
    'income',
    'gender_O'
]

df.drop(columns=cols_to_drop, inplace=True)

In [41]:
# Check one more time - Looking good
# Run again to check
# checking for Multicollinearity
# Using the Variance Inflation Factor (VIF) to remove redundant features.
feature_columns = df.columns.tolist()
X = df[feature_columns].copy()
X = X.drop(columns=['offer_id', 'customer_id', 'completed_flag_final'])

X = add_constant(X)  # Add constant term

vif_data = pd.DataFrame()

vif_data["feature"] = X.columns

vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data.sort_values(by="VIF", ascending=False))

                             feature        VIF
0                              const  82.472241
2                           duration   4.926039
9           offer_type_informational   2.742354
8                offer_type_discount   2.644197
5   channels_email,mobile,social,web   2.635192
7                 channels_email,web   2.163119
6          channels_email,mobile,web   1.912332
10                   income_band_Low   1.518070
11                income_band_Medium   1.484249
1                                age   1.324556
4                           gender_M   1.209174
3                        member_days   1.002642


### Model
Building a model that predicts whether or not someone will respond to an offer using logistic regression.

# Steps
* Split data in to train, test
* Evaluate using metrics like accuracy, precision, recall, and F1-score.

In [43]:
# Selecting Features and target
X = df[['age', 'duration', 'member_days', 'gender_M', 'channels_email,mobile,social,web', 'channels_email,mobile,web', 'channels_email,web',
       'offer_type_discount', 'offer_type_informational', 'income_band_Low', 'income_band_Medium']]
y = df['completed_flag_final']

In [44]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fit logistic regression
model = LogisticRegression(max_iter=500)
model.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=500)

#### Evaluate model performance

In [46]:
y_pred = model.predict(X_test_scaled)
y_proba = model.predict_proba(X_test_scaled)[:, 1]  # For ROC AUC

In [47]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_proba))

Accuracy: 0.7419459811259356
Precision: 0.6711769759450171
Recall: 0.655686109945447
F1 Score: 0.6633411165357673
ROC AUC Score: 0.8266032070776825


My model performs okay, especially considering the ROC AUC of 0.827. F1 score of 0.663 suggests a decent tradeoff between catching positives and avoiding false alarms. There is still room for improvement however.

In [49]:
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Confusion Matrix:
 [[5995 1531]
 [1641 3125]]

Classification Report:
               precision    recall  f1-score   support

         0.0       0.79      0.80      0.79      7526
         1.0       0.67      0.66      0.66      4766

    accuracy                           0.74     12292
   macro avg       0.73      0.73      0.73     12292
weighted avg       0.74      0.74      0.74     12292



## Try applying SMOTE and switching to a random forest to see if that improves model performance. 

In [51]:
# 1. Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Apply SMOTE to training data only
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# 3. Fit Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_resampled, y_resampled)

# 4. Predict and evaluate
y_pred = rf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[5936 1590]
 [1618 3148]]
              precision    recall  f1-score   support

         0.0       0.79      0.79      0.79      7526
         1.0       0.66      0.66      0.66      4766

    accuracy                           0.74     12292
   macro avg       0.73      0.72      0.72     12292
weighted avg       0.74      0.74      0.74     12292



### Try adding SMOTE to the logistic regression

In [53]:
# 1. Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Apply SMOTE to training data
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# 3. Create a pipeline with scaling and logistic regression
model = make_pipeline(StandardScaler(), LogisticRegression(max_iter=500))

# 4. Fit on resampled data
model.fit(X_train_resampled, y_train_resampled)

# 5. Predict on the original test set
y_pred = model.predict(X_test)

# 6. Evaluate
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[5129 2397]
 [ 898 3868]]
              precision    recall  f1-score   support

         0.0       0.85      0.68      0.76      7526
         1.0       0.62      0.81      0.70      4766

    accuracy                           0.73     12292
   macro avg       0.73      0.75      0.73     12292
weighted avg       0.76      0.73      0.74     12292



### Try tuning the parameters

In [55]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply SMOTE to only the training data
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Scale the data (fit scaler on resampled train set)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)

# Parameter grid for Random Forest
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
}

# Initialize RandomForestClassifier
rf = RandomForestClassifier(random_state=42)

# GridSearchCV with 5-fold CV and f1 scoring
grid = GridSearchCV(rf, param_grid, cv=5, scoring='f1', n_jobs=-1)
grid.fit(X_train_scaled, y_train_resampled)

# Print best parameters and best f1 score from CV
print("Best params:", grid.best_params_)
print("Best F1 score:", grid.best_score_)

# Train final model using best params on full resampled training data
best_model = RandomForestClassifier(**grid.best_params_, random_state=42)
best_model.fit(X_train_scaled, y_train_resampled)

Best params: {'bootstrap': False, 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best F1 score: 0.8030592279529095


RandomForestClassifier(bootstrap=False, max_depth=10, n_estimators=200,
                       random_state=42)

In [56]:
y_pred = best_model.predict(X_test_scaled)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, best_model.predict_proba(X_test_scaled)[:, 1]))

[[5930 1596]
 [1219 3547]]
              precision    recall  f1-score   support

         0.0       0.83      0.79      0.81      7526
         1.0       0.69      0.74      0.72      4766

    accuracy                           0.77     12292
   macro avg       0.76      0.77      0.76     12292
weighted avg       0.78      0.77      0.77     12292

ROC AUC: 0.8624095442415936


Summary:
The Random Forest model did better than logistic regression overall. It had higher accuracy, better precision and F1-score for the positive class, and a stronger ROC AUC—so it's separating the classes more effectively.

Logistic regression had a slightly better recall for class 1, meaning it caught more of the positives, but it also made more false positive predictions. So, while it was more sensitive, it wasn’t as precise.

Bottom Line:
Random Forest is the best model I've got so far. It strikes a good balance between precision, recall, and overall performance. I might play around with tuning it more or try out other models like XGBoost or LightGBM to see if I can push it even further.

#### Save to Pickle

In [59]:
# Save the model and scaler with pickle
with open('random_forest_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)